In [1]:
import os, sys, pdb
import numpy as np
import vis_utils
from tqdm import tnrange, tqdm_notebook

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
model_json = vis_utils.json_load('models.json')

In [63]:
model_type = 'cifar'
top_level= [_dir for _dir in os.listdir(model_type) if not _dir.startswith('.')]
dir_paths = {entry:os.path.join(model_type, entry) for entry in top_level}
jsons = os.listdir(dir_paths['jsons'])
model_dirs = [os.path.splitext(_item)[0] for _item in jsons]
classes = os.listdir(os.path.join(dir_paths['activations'], model_dirs[0])+'/epoch14')

# Overview
Each model has its own activation data that was collected beforehand. We will go into these activations, and compute the max activations for each class, using their data.

In [65]:
model_dict={}
for m_idx in tqdm_notebook(range(len(model_dirs)), desc="Models"):
    class_dict={}
    for idx in tqdm_notebook(range(len(classes)), desc="classes", leave=False):
        classes_activations_path = os.path.join(dir_paths['activations'], model_dirs[m_idx])+'/epoch14/'+classes[idx]
        class_activations = os.listdir(classes_activations_path)
        # Set up the sorting and activation dictionaries
        fidx = 0
        activation_json = vis_utils.json_load(os.path.join(classes_activations_path, class_activations[fidx]))
        conv_layers = {str(item):len(activation_json[item]['sorted']) for item in activation_json}
        sort_dict = {layers:[0]*conv_layers[layers] for layers in conv_layers}
        act_dict = {layers:[0]*conv_layers[layers] for layers in conv_layers}
        # Perform the calculations here
        for fidx in tqdm_notebook(range(len(class_activations)), desc="Files", leave=False):
            activation_json = vis_utils.json_load(os.path.join(classes_activations_path, class_activations[fidx]))
            sort_dict = {layer : [x+y for x,y in zip(sort_dict[layer],activation_json[layer]['sorted'])] for layer in conv_layers}
            act_dict = {layer : [x+y for x,y in zip(act_dict[layer],activation_json[layer]['activations'])] for layer in conv_layers}
        class_dict[classes[idx]] = {'activations':dict(act_dict), 'sorted':dict(sort_dict)}
    model_dict[model_dirs[m_idx]] = dict(class_dict)

In [70]:
with open('activation_sums.json', 'w') as _compile:
    json.dump(model_dict, _compile)

# Analysis

## Activation Details

In [140]:
for model_item in model_dirs:
    print model_item.upper()
    cifar_1 = model_dict[model_item]
    conv_layers = [top_level for top_level in model_dict[model_item]['horse']['sorted']]
    conv_layers.sort()
    for _class in cifar_1:
        print '    Class: ', _class
        for layers in conv_layers:
            top_kernels = np.argsort(np.argsort(cifar_1[_class]['sorted'][layers]))[::-1][0:5]
            top_act = np.array(cifar_1[_class]['activations'][layers])[top_kernels.astype("int")]
            print "        Top kernels in  : ", layers.ljust(8), top_act
        print ""
        print ""    

CIFAR_6
    Class:  horse
        Top kernels in  :  conv_1   [  7.528   0.106  20.892   0.055  84.947]
        Top kernels in  :  conv_10  [ 24.138  13.935  24.519  16.661   4.115]
        Top kernels in  :  conv_2   [  77.83     4.807    0.     116.313    2.25 ]
        Top kernels in  :  conv_3   [ 283.297   52.859    0.       0.      17.081]
        Top kernels in  :  conv_4   [  15.569   70.755  160.467    0.       0.   ]
        Top kernels in  :  conv_5   [ 114.374    4.946  138.599    0.19    28.897]
        Top kernels in  :  conv_6   [  25.47    42.392   51.179   21.195  291.194]
        Top kernels in  :  conv_7   [ 219.526   13.147    1.299  109.387  123.243]
        Top kernels in  :  conv_8   [ 39.61    0.      6.826  68.049   0.   ]
        Top kernels in  :  conv_9   [  0.     59.163   6.452  66.165   2.275]


    Class:  automobile
        Top kernels in  :  conv_1   [ 16.316   0.128   2.785   0.223  19.545]
        Top kernels in  :  conv_10  [ 31.562  18.592   5.46  

## Kernel Details

In [115]:
for model_item in model_dirs:
    print model_item.upper()
    cifar_1 = model_dict[model_item]
    conv_layers = [top_level for top_level in model_dict[model_item]['horse']['sorted']]
    conv_layers.sort()
    for _class in cifar_1:
        print '    Class: ', _class
        for layers in conv_layers:
            top_kernels = np.argsort(np.argsort(cifar_1[_class]['sorted'][layers]))[::-1][0:5]
            print "        Top kernels in  : ", layers.ljust(8), "".join([str(i).rjust(5) for i in top_kernels])
        print ""
        print ""    

CIFAR_6
    Class:  horse
        Top kernels in  :  conv_1      31   28    8    3   11
        Top kernels in  :  conv_10      0    1    2    3   12
        Top kernels in  :  conv_2       0   28   26   24    5
        Top kernels in  :  conv_3       1    2   16    8    6
        Top kernels in  :  conv_4       4    9    8    5    6
        Top kernels in  :  conv_5       1    0    2    8   11
        Top kernels in  :  conv_6      22    9   16   24   23
        Top kernels in  :  conv_7      17   19    6    0    2
        Top kernels in  :  conv_8       1   17   21   24   26
        Top kernels in  :  conv_9      10   14   15   28   20


    Class:  automobile
        Top kernels in  :  conv_1      31   27   10    2    8
        Top kernels in  :  conv_10      0    5   15    9   22
        Top kernels in  :  conv_2       1   10   11   28   25
        Top kernels in  :  conv_3       1    2   19   11    7
        Top kernels in  :  conv_4       4   12   20   22   21
        Top kernels